In [1]:
import pickle 
tid_clusters_list = pickle.load(open('tid_clusters_list.p','rb'))

In [2]:
count = 0
for tid, clusters in tid_clusters_list.items():
    for cluster in clusters: 
        count +=len(cluster['torsions'])
print(f'# input entries: {count}')

# input entries: 1085


In [3]:
# copied from https://github.com/openforcefield/qca-dataset-submission/blob/master/submissions/2021-03-23-OpenFF-Amide-Torsion-Set-v1.0/generate-dataset.ipynb
import logging
import warnings
from pprint import pprint

import numpy as np
from openeye import oechem
from openff.qcsubmit.common_structures import TorsionIndexer
from openff.qcsubmit.factories import TorsiondriveDatasetFactory
from openff.qcsubmit.workflow_components import StandardConformerGenerator, EnumerateStereoisomers, EnumerateProtomers
from openforcefield.topology import Molecule
from tqdm import tqdm

/Users/hyesu/anaconda3/envs/new-qcsubmit/lib/python3.7/site-packages/sqlalchemy/sql/coercions.py:308: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  "implicitly coercing SELECT object to scalar subquery; "


In [4]:
# Warnings that tell us we have undefined stereo and charged molecules
logging.getLogger("openforcefield").setLevel(logging.ERROR)
warnings.simplefilter("ignore")

In [6]:
molecules = []

for tid, clusters in tqdm(tid_clusters_list.items()):
    for cluster in clusters: 
        torsions = cluster['torsions']
        for torsion in torsions: 
            smiles_pattern = torsion['mol_index']
            dihedrals = torsion['indices']
            molecule: Molecule = Molecule.from_smiles(smiles_pattern,allow_undefined_stereo=True)
            molecule = molecule.canonical_order_atoms()
            try:
                molecule.generate_conformers(n_conformers=10)
            except Exception:
                print(f"Skipping {smiles_pattern} - OMEGA error.")
                continue
            try:
                molecule.enumerate_stereoisomers(max_isomers=1)
            except Exception:
                print(f"Skipping {smiles_pattern} - stereo enumeration error.")
                continue

            torsion_indexer = TorsionIndexer()
            torsion_indexer.add_torsion(dihedrals, (-165, 180), True)

            molecule.properties["dihedrals"] = torsion_indexer
            molecules.append(molecule)

100%|██████████| 167/167 [06:27<00:00,  2.32s/it]


In [7]:
len(molecules)

1085

In [8]:
dataset_factory = TorsiondriveDatasetFactory()

# dataset_factory.add_workflow_component(
#     StandardConformerGenerator(max_conformers=10, rms_cutoff=0.1, clear_existing=True)
# )
dataset = dataset_factory.create_dataset(
    dataset_name="OpenFF Gen3 Torsion Set v1.0",
    tagline="OpenFF Gen3 Torsion Set v1.0",
    description="This dataset is a simple-molecule-only dataset, a candidate of Sage torsion parameter training set",
    molecules=molecules,
)

dataset.metadata.submitter = "hyesujang"
dataset.metadata.long_description_url = (
    "https://github.com/openforcefield/qca-dataset-submission/tree/master/"
    "submissions/"
    "2021-04-09-OpenFF-Gen3-Torsion-Set-v1.0"
)

Preparation                   : 100%|█████████| 888/888 [00:13<00:00, 63.52it/s]


In [9]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of torsion drives         ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    oemol = molecule.to_openeye()
    mass = oechem.OECalculateMolecularWeight(oemol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

Number of unique molecules        888
Number of filtered molecules      0
Number of torsion drives          889
Number of conformers min mean max 1   2.61 12
Mean molecular weight: 131.31
Max molecular weight: 433.68
Charges: [0.0, 1.0]


In [10]:
pprint(dataset.metadata.dict())

{'collection_type': 'TorsiondriveDataset',
 'creation_date': datetime.date(2021, 4, 19),
 'dataset_name': 'OpenFF Gen3 Torsion Set v1.0',
 'elements': {'Br', 'Cl', 'N', 'C', 'O', 'F', 'P', 'H', 'S'},
 'long_description': 'This dataset is a simple-molecule-only dataset, a '
                     'candidate of Sage torsion parameter training set',
 'long_description_url': HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-04-09-OpenFF-Gen3-Torsion-Set-v1.0', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2021-04-09-OpenFF-Gen3-Torsion-Set-v1.0'),
 'short_description': 'OpenFF Gen3 Torsion Set v1.0',
 'submitter': 'hyesujang'}


In [11]:
for spec, obj in dataset.qc_specifications.items():
    print("Spec:", spec)
    pprint(obj.dict())

Spec: default
{'basis': 'DZVP',
 'implicit_solvent': None,
 'method': 'B3LYP-D3BJ',
 'program': 'psi4',
 'spec_description': 'Standard OpenFF optimization quantum chemistry '
                     'specification.',
 'spec_name': 'default',
 'store_wavefunction': 'none'}


In [12]:
pprint(dataset.scf_properties)

[<SCFProperties.Dipole: 'dipole'>,
 <SCFProperties.Quadrupole: 'quadrupole'>,
 <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>,
 <SCFProperties.MayerIndices: 'mayer_indices'>]


In [13]:
dataset.export_dataset("dataset.json.bz2")
dataset.molecules_to_file("dataset.smi", "smi")

dataset.visualize("dataset.pdf", columns=8)